In [1]:
import pandas as pd
from rdkit import Chem
import numpy as np
from pathlib import Path as pt
from time import  perf_counter

In [3]:
from xgboost import XGBRegressor, __version__ as xgboost_version

from catboost import CatBoostRegressor, __version__ as catboost_version

from lightgbm import LGBMRegressor, __version__ as lightgbm_version
print(f'{xgboost_version=}, {catboost_version=}, {lightgbm_version=}')

/Users/aravindhnivas/anaconda3/envs/umdapy/lib/python3.9/site-packages/dask/dataframe/_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(


xgboost_version='2.1.0', catboost_version='1.2.5', lightgbm_version='4.5.0'


In [72]:
import numpy as np
from pathlib import Path as pt

# for processing
import pandas as pd
from sklearn import metrics, __version__ as sklearn_version

print(f"Using scikit-learn version {sklearn_version}")

# models
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, HuberRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor
from sklearn.gaussian_process import GaussianProcessRegressor, kernels

# explicitly require this experimental feature
from sklearn.experimental import enable_halving_search_cv  # noqa
from sklearn.model_selection import (
    KFold,
    cross_val_score,
    train_test_split,
    GridSearchCV,
    RandomizedSearchCV,
    HalvingGridSearchCV,
    HalvingRandomSearchCV,
)
from sklearn.preprocessing import StandardScaler, QuantileTransformer

from dask_ml.model_selection import (
    GridSearchCV as DaskGridSearchCV,
    RandomizedSearchCV as DaskRandomizedSearchCV,
)

# for saving models
from joblib import dump
from sklearn.utils import resample

from dask.diagnostics import ProgressBar

import json
from scipy.optimize import curve_fit

Using scikit-learn version 1.5.1


In [34]:
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [7]:
def linear(x, m, c):
    return m * x + c

In [13]:
loc = pt('/Users/aravindhnivas/Documents/test-codes/umda/ML properties/')
mp_csv_file = loc / 'melting_points_updated.csv'
mp_csv_file.exists()

True

In [15]:
df = pd.read_csv(mp_csv_file, index_col=0)
df

,SMILES,Melting point (C),MP (processed)
0,COP(=S)(OC)OC1=CC=C(C=C1)SC2=CC=C(C=C2)OP(=S)(...,31.6(5),31.60
1,CC(C)C1=CC2=CCC3C(C2CC1)(CCCC3(C)C(=O)O)C,173.5,173.50
2,CC1=CC(=O)CC(C1(C=CC(=CC(=O)O)C)O)(C)C,160,160.00
3,COC1=CC=C(C=C1)C2=CC(=O)C3=C(C=C(C=C3O2)O)O,263,263.00
4,CCCC(=O)NC1=CC(=C(C=C1)OCC(CNC(C)C)O)C(=O)C,121,121.00
...,...,...,...
7976,CCCCN(CCCC)C(=S)[S-].CCCCN(CCCC)C(=S)[S-].[Zn+2],138,138.00
7977,C[CH2-].C[CH2-].[Zn+2],-33.34(2),-33.34
7979,CC(=CC(=O)C)O.CC(=CC(=O)C)O.[Zn],127.3(2),127.30
7980,CN(C)C(=S)[S-].CN(C)C(=S)[S-].[Zn+2],250,250.00


In [36]:
embedding_file = loc / 'melting_points_updated_SMILES_VICGAE_embeddings.npy'
embedding_file.exists()

True

In [61]:
X = np.load(embedding_file, allow_pickle=True)
X.shape

(7437, 32)

In [62]:
y = df['MP (processed)'].values
y.shape

(7437,)

In [63]:
invalid_indices = [i for i, arr in enumerate(X) if np.any(arr == 0)]
valid_mask = np.ones(len(X), dtype=bool)  # Initially, mark all as valid
valid_mask[invalid_indices] = False  # Mark invalid indices as False
X = X[valid_mask]  # Keep only the rows that are marked as True in the valid_mask
y = y[valid_mask]
X.shape, y.shape

((7123, 32), (7123,))

### Outlier Detection and Handling:

In [46]:
# Assuming X is your feature matrix and y is your target variable
outlier_detector = IsolationForest(contamination=0.1, random_state=42)
outlier_labels = outlier_detector.fit_predict(np.column_stack((X, y.reshape(-1, 1))))

# Filter out outliers
X_filtered = X[outlier_labels == 1]
y_filtered = y[outlier_labels == 1]

print(f"Original data points: {len(X)}")
print(f"Data points after outlier removal: {len(X_filtered)}")

X = X_filtered
y = y_filtered

X.shape, y.shape

Original data points: 7123
Data points after outlier removal: 6410


((6410, 32), (6410,))

### Feature Importance Analysis

In [51]:
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X, y)

feature_importance = rf.feature_importances_
for i, importance in enumerate(feature_importance):
    print(f"Feature {i}: {importance}")

# Keep only the top 20 most important features
top_features = np.argsort(feature_importance)[-20:]
X_top = X[:, top_features]
X_top.shape

Feature 0: 0.026268399787470552
Feature 1: 0.018105292756951614
Feature 2: 0.011940207771889904
Feature 3: 0.012791157071970144
Feature 4: 0.03963327507317398
Feature 5: 0.011271968959328369
Feature 6: 0.019415413930164147
Feature 7: 0.01697938582692336
Feature 8: 0.04274030623071726
Feature 9: 0.031312935677226636
Feature 10: 0.03171787752793294
Feature 11: 0.2878232329958599
Feature 12: 0.011666538337412482
Feature 13: 0.01641038118149706
Feature 14: 0.01191087854831993
Feature 15: 0.013160059813697248
Feature 16: 0.01838959926596305
Feature 17: 0.020449902079533108
Feature 18: 0.015900952533121362
Feature 19: 0.013860397224998915
Feature 20: 0.03407904908125294
Feature 21: 0.07120347969433995
Feature 22: 0.01415358888105597
Feature 23: 0.012287965239397158
Feature 24: 0.016079750032623942
Feature 25: 0.05872488126774851
Feature 26: 0.012489109720671769
Feature 27: 0.0424379140439085
Feature 28: 0.012599792399812357
Feature 29: 0.01613116309120003
Feature 30: 0.018770030914448333
Fea

(6410, 20)

### Data Transformations:

In [64]:
scaler = StandardScaler()
def std_scale_data(ydata, inverse=False):
    if inverse:
        return scaler.inverse_transform(ydata.reshape(-1, 1)).flatten()
    return scaler.fit_transform(y.reshape(-1, 1)).flatten()

qt = QuantileTransformer(n_quantiles=1000, output_distribution='normal')
def qt_scale_data(ydata, inverse=False):
    if inverse:
        return qt.inverse_transform(ydata.reshape(-1, 1)).ravel()
    return qt.fit_transform(y.reshape(-1, 1)).ravel()


In [68]:
# scaling_fn = std_scale_data
scaling_fn = qt_scale_data
y_scaled = scaling_fn(y)
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, y_scaled, test_size=float(test_size))
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((5698, 32), (1425, 32), (5698,), (1425,))

In [89]:
np.isin(np.arange(len(y_test)), [0, 1, 2, 3, 4, 5, 6, 7, 8, 9])[0]

True

In [93]:
def main(model, CV=False):

    start = perf_counter()
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_pred = scaling_fn(y_pred, inverse=True)
    y_test_original = scaling_fn(y_test, inverse=True)
    
    # Identify data points with large errors
    residuals = y_test_original - y_pred
    large_error_indices = np.where(np.abs(residuals) > 2 * np.std(residuals))[0]
    print(f"Number of large error data points: {len(large_error_indices)}")
    
    #  keeps only the elements whose indices are not in large_error_indices.
    # y_test_original = y_test_original[~np.isin(np.arange(len(y_test_original)), large_error_indices)]
    # y_pred = y_pred[~np.isin(np.arange(len(y_pred)), large_error_indices)]
    
    r2 = metrics.r2_score(y_test_original, y_pred)
    mse = metrics.mean_squared_error(y_test_original, y_pred)
    rmse = np.sqrt(mse)
    mae = metrics.mean_absolute_error(y_test_original, y_pred)

    # print(f"R2: {r2:.4f}, MSE: {mse:.4f}, RMSE: {rmse:.4f}, MAE: {mae:.4f}")

    pop, _ = curve_fit(linear, y_test_original, y_pred)
    y_linear_fit = linear(y_test_original, *pop)

    fig, ax = plt.subplots(figsize=(8, 8))
    ax.plot(y_test_original, y_pred, 'o', label=model.__class__.__name__)
    ax.plot(y_test_original, y_linear_fit, label='Linear fit')
    ax.set_xlabel('True MP (°C)')
    ax.set_ylabel('Predicted MP (°C)')
    ax.legend()
    ax.title.set_text(f"R$^2$ = {r2:.2f}, MSE = {mse:.2f}, RMSE = {rmse:.2f}, MAE = {mae:.2f}")
    
    if CV:
        cv_scores = cross_val_score(model, X, y, cv=10, scoring="r2")
        print(f"Mean CV R2 score: {cv_scores.mean():.2f} (+/- {cv_scores.std() * 2:.2f})")
        r2_label = f"R$^2$: {r2:.2f}, CV R$^2$: {cv_scores.mean():.2f} (+/- {cv_scores.std() * 2:.2f})"
        ax.title.set_text(f"{r2_label}, MSE = {mse:.2f}, RMSE = {rmse:.2f}, MAE = {mae:.2f}")
    
    print(f"Time taken: {perf_counter() - start:.2f} seconds")
    return model

In [73]:
estimators = [
    ('rf', RandomForestRegressor(n_estimators=100, random_state=42)),
    ('xgb', XGBRegressor(random_state=42)),
    ('svr', SVR())
]

stacking_regressor = StackingRegressor(
    estimators=estimators,
    final_estimator=LinearRegression()
)

# stacking_regressor.fit(X_train, y_train)

In [97]:
# %matplotlib widget
plt.close('all')
# CV = True
CV = False
# _ = main(XGBRegressor(n_jobs=-1, verbosity=0), CV=CV)
_ = main(CatBoostRegressor(), CV=CV)
# _ = main(LGBMRegressor(metric='rmse'), CV=CV)
# _ = main(HuberRegressor(), CV=CV)
# _ = main(stacking_regressor, CV=CV)

TypeError: main() got an unexpected keyword argument 'verbose'

In [24]:
# Define parameter grids for each model
lgbm_param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.3],
    'max_depth': [3, 5, 7],
    'num_leaves': [31, 63, 127]
}

catboost_param_grid = {
    'iterations': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.3],
    'depth': [4, 6, 8],
    'l2_leaf_reg': [1, 3, 5, 7]
}

xgb_param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.3],
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 3, 5]
}

# Function to perform grid search and return best model
def grid_search_model(model, param_grid):
    grid_search = GridSearchCV(model, param_grid, cv=3, n_jobs=-1)
    grid_search.fit(X_train, y_train)
    return grid_search.best_estimator_, grid_search.best_params_

# Function to evaluate model
def evaluate_model(model):
    y_pred = model.predict(X_test)
    mse = metrics.mean_squared_error(y_test, y_pred)
    r2 = metrics.r2_score(y_test, y_pred)
    return mse, r2

In [25]:
# Fine-tune LightGBM
print("Fine-tuning LightGBM...")
lgbm_model, lgbm_best_params = grid_search_model(LGBMRegressor(random_state=42), lgbm_param_grid)
lgbm_mse, lgbm_r2 = evaluate_model(lgbm_model)
print("LightGBM - Best parameters:", lgbm_best_params)
print(f"LightGBM - MSE: {lgbm_mse:.4f}, R2: {lgbm_r2:.4f}")

# Fine-tune CatBoost
print("Fine-tuning CatBoost...")
catboost_model, catboost_best_params = grid_search_model(CatBoostRegressor(random_state=42, verbose=False), catboost_param_grid)
catboost_mse, catboost_r2 = evaluate_model(catboost_model)
print("\nCatBoost - Best parameters:", catboost_best_params)
print(f"CatBoost - MSE: {catboost_mse:.4f}, R2: {catboost_r2:.4f}")

# Fine-tune XGBoost
print("Fine-tuning XGBoost...")
xgb_model, xgb_best_params = grid_search_model(XGBRegressor(random_state=42), xgb_param_grid)
xgb_mse, xgb_r2 = evaluate_model(xgb_model)

print("\nXGBoost - Best parameters:", xgb_best_params)
print(f"XGBoost - MSE: {xgb_mse:.4f}, R2: {xgb_r2:.4f}")

Fine-tuning LightGBM...
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003839 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8160
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001818 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8160
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.017517 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8160
[LightGBM] [Info] Number of data points in the train set: 3799, number of used features: 32
[LightGBM] [Info] Number of data points in the train set: 3799, number of used features: 32
[LightGBM] [Info] Number of data points in the train set: 3799, number of used features: 32
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.026197 seconds.
You can 

In [26]:
print("LightGBM - Best parameters:", lgbm_best_params)
print(f"LightGBM - MSE: {lgbm_mse:.4f}, R2: {lgbm_r2:.4f}")

print("\nCatBoost - Best parameters:", catboost_best_params)
print(f"CatBoost - MSE: {catboost_mse:.4f}, R2: {catboost_r2:.4f}")

print("\nXGBoost - Best parameters:", xgb_best_params)
print(f"XGBoost - MSE: {xgb_mse:.4f}, R2: {xgb_r2:.4f}")

LightGBM - Best parameters: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 300, 'num_leaves': 63}
LightGBM - MSE: 0.3070, R2: 0.6629

CatBoost - Best parameters: {'depth': 8, 'iterations': 300, 'l2_leaf_reg': 5, 'learning_rate': 0.1}
CatBoost - MSE: 0.3010, R2: 0.6695

XGBoost - Best parameters: {'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 3, 'n_estimators': 200}
XGBoost - MSE: 0.3175, R2: 0.6514


In [ ]:
# grid search
xgb_model = XGBRegressor(
    n_jobs=-1, 
    verbosity=1,
)

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.7, 0.9],
    'colsample_bytree': [0.7, 0.9],
    'gamma': [0, 0.1],
    'reg_alpha': [0, 1],
    'reg_lambda': [0, 1]
}
grid_search = GridSearchCV(
# grid_search = RandomizedSearchCV(
# grid_search = HalvingRandomSearchCV(
# grid_search = HalvingGridSearchCV(
    xgb_model, param_grid, cv=3, n_jobs=-1, 
    # verbose=1, 
    # n_iter=100,  # number of parameter settings that are sampled
)

# Ensure X_train and y_train are correctly formatted
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")


# Check for NaN values
print(f"X_train contains NaN: {np.isnan(X_train).any()}")
print(f"y_train contains NaN: {np.isnan(y_train).any()}")

try:
    grid_search.fit(X_train, y_train)
except ValueError as e:
    print(f"ValueError: {e}")

# grid_results = pd.DataFrame(grid_search.cv_results_)
# grid_results = grid_results.sort_values(by='rank_test_score')
# grid_results.to_csv(loc / f'grid_search_results_{xgb_model.__class__.__name__}.csv', index=False)

# Access the best estimator if fit is successful
if grid_search.best_estimator_:
    best_model = grid_search.best_estimator_
    _ = main(best_model)

In [ ]:
grid_search.best_params_, grid_search.best_score_

In [ ]:
grid_results = pd.DataFrame(grid_search.cv_results_)
grid_results = grid_results.sort_values(by='rank_test_score')
grid_results.to_csv(loc / f'grid_search_results_{xgb_model.__class__.__name__}.csv', index=False)

In [ ]:
xgb_model.get_params()

Original data points: 7123
Data points after outlier removal: 6410


In [33]:
outlier_labels

array([ 1, -1,  1, ..., -1,  1,  1])